# Save Load and Predict using Iris Flower Dataset

In [9]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

seed = 7
numpy.random.seed(seed)

dataframe = pandas.read_csv("02-iris.csv", delimiter=',', header=None)
dataset = dataframe.values

X = dataset[:,0:4]
y = dataset[:,4]

encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)

dummy_y = np_utils.to_categorical(encoded_y)

def baseline_model():
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 97.33% (4.42%)


In [10]:
estimator.fit(X,y)

#save model as Json
model_json = estimator.model.to_json()
with open("model_iris.json", "w") as json_file:
    json_file.write(model_json)

#save the weights as hdf5
estimator.model.save_weights("model_iris.h5")

print("Saved the model to the computer")

Saved the model to the computer


In [11]:
from keras.models import model_from_json

#load the json from the file and recreate the model
json_file = open('model_iris.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

#load the weights from hdf5 file
loaded_model.load_weights("model_iris.h5")
print("Loaded model from the computer")

Loaded model from the computer


In [13]:
from numpy import array
Xnew = array([[6.3,2.5,5,1.9]])

ynew = loaded_model.predict_classes(Xnew)

print("X=%s, Predicted=%s" % (Xnew[0], ynew[0]))
print("X=%s, Predicted=%s" % (Xnew[0], numpy.unique(y)[ynew[0]]))

X=[6.3 2.5 5.  1.9], Predicted=2
X=[6.3 2.5 5.  1.9], Predicted=Iris-virginica
